In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from scipy.sparse import hstack


In [ ]:
df = pd.read_csv("data\combined_df.csv")
df

In [ ]:
#min max scaler for rating and year
scaler = MinMaxScaler()
df['Rating Scale'] = scaler.fit_transform(df[['Average Rating']])
df['Year Scale'] = scaler.fit_transform(df[['Publish Date']])



#tf idf for synop + reviews
#could also test combining the reviews and synopsis and doing one big tfidf, but I prefer doing seperate to (hopefully make it more accurate)
vectorizer = TfidfVectorizer(ngram_range=(1,2))
reviews_tfidf = vectorizer.fit_transform(df['Proccessed Review'])
synopsis_tfidf = vectorizer.fit_transform(df['Proccessed Synopsis'])

#genres, book titles authors
if df['Genres'] is str:
    df['Genres'] = df['Genres'].str.split("'")
    print(df['Genres'])
encoder = OneHotEncoder(handle_unknown='ignore') #method for vectorizing the the following: 
encoded_genres = encoder.fit_transform(df['Genres'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('Genres'))
encoded_titles = encoder.fit_transform(df[['Book Title']])
encoded_authors = encoder.fit_transform(df[['Author']])
encoded_features = hstack([encoded_titles, encoded_authors, encoded_genres])


combined_features = hstack([encoded_features, df[['Rating Scale', 'Year Scale']], reviews_tfidf, synopsis_tfidf])

print(combined_features.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

similarity_matrix = cosine_similarity(combined_features)
print(similarity_matrix.shape)
np.save('arrays/combined_cosine_sim.npy', similarity_matrix)
